**Angelo Joaquin B. Alvarez (210295)**

# Homework 1

*Due date:* February 3, 2024 (Saturday) at 8 PM on CodePost

This homework is designed to get you started in implementing some cryptographic algorithms in Python.
This should be easy enough for you to do solo, but it doesn't hurt if you want to work with a partner.
These exercises are important in the sense that they serve as stepping stones to future attacks, and most likely you'll need your code for these for the final contest.
If you can finish this homework on time, then you'll very likely do well in the final contest!

Much of classical crypto operates on alphabetic strings (strings only containing letters) while modern 
crypto deals excusively with binary strings.
Despite that crucial difference, we may see some familiar parallels between historical ciphers like 
Caesar or Vigenère and relatively modern ones like the XOR cipher and the one-time pad.

This homework has 32 points in total, but will be divided by 30 to get the final percentage. Final percentages are capped at 100%.

Please be guided on the policies regarding late submissions, regrading, and collaboration.
If any, please direct all your questions and clarifications about this homework in the `#hw1-help` channel on the Discord server.

## Dealing with binary data in Python

Python supports *byte literals* (I just call them "bytestrings" for convenience), string-like sequences that are prefixed by `b`.

In [1]:
b'This is a byte literal'

b'This is a byte literal'

In [2]:
'Meanwhile this is a string literal'

'Meanwhile this is a string literal'

A bytestring is not the same as its string counterpart...

In [3]:
'some_string' == b'some_string'

False

...because they have different types.

In [4]:
print(type('some_string'))
print(type(b'some_string'))

<class 'str'>
<class 'bytes'>


The usual string operations work with bytestrings.

In [5]:
test = b'this is a long bytestring'
print(test[3:10])   # slicing
print(test[4])      # get the 4th byte (returns an int)

b's is a '
32


In [6]:
another = b' and anotha one!'
print(test + another)   # concatenating bytestrings
print(another * 10)     # n-fold concatenation

b'this is a long bytestring and anotha one!'
b' and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one! and anotha one!'


We can use the following libraries to convert between encodings (hex and Base64).

In [7]:
from binascii import hexlify, unhexlify

In [8]:
from base64 import b64encode, b64decode

In [9]:
hexlify(b'this is gonna be hex soon')

b'7468697320697320676f6e6e612062652068657820736f6f6e'

In [10]:
unhexlify(b'7468697320697320686578206e6f206d6f7265')

b'this is hex no more'

In [11]:
b64encode(b'Base64 this thing!')

b'QmFzZTY0IHRoaXMgdGhpbmch'

In [12]:
b64decode(b'YmFzZTY0IGlzIG5vdCBlbmNyeXB0aW9uIQ==')

b'base64 is not encryption!'

## Some reminders

You are not allowed to use additional libraries (even within the Python standard library) other than those explicitly used here, though you may implement additional functions of your own.

**Very important:** Always work with raw bytes, never with encoded strings.

The following pages from the Python documentation may be helpful:
- https://docs.python.org/3/library/stdtypes.html#bytes-objects
- https://docs.python.org/3/library/binascii.html
- https://docs.python.org/3/library/base64.html

## 1-1. Hex to Base64 [2 pts]

Write a function called `hex_to_b64` to convert a hex-encoded bytestring into Base64. For example, the string
```
48656c7021204920676f7420584f52206579657320616e642069742773206b696c6c696e67206d65
```
should output
```
SGVscCEgSSBnb3QgWE9SIGV5ZXMgYW5kIGl0J3Mga2lsbGluZyBtZQ==
```

In [13]:
def hex_to_b64(h):
    return b64encode(unhexlify(h))

In [14]:
# if your function works properly, no error should appear when running this line
assert hex_to_b64(b'48656c7021204920676f7420584f52206579657320616e642069742773206b696c6c696e67206d65') == b'SGVscCEgSSBnb3QgWE9SIGV5ZXMgYW5kIGl0J3Mga2lsbGluZyBtZQ=='

## 1-2. XOR'ing two bytestrings [4 pts]

Write a function called `xor_bytes` that takes two bytestrings of equal length and outputs their XOR. For example, if I have a bytestring
```
49207374696c6c206861766520584f522065796573206e6f772077686174
```
after hex-decoding it, and when I XOR it against
```
3a4f1e11060209001b04180100302a3e5045141c5345170a040016292955
```
it should output
```
736f6d656f6e652073656e642068656c70206d7920657965732061414821
```

In [15]:
def xor_bytes(a, b):
    if len(a) == len(b):
        results = []
        for i in range(0, len(a)):
            results.append(a[i] ^ b[i])
        return bytes(results)
    else:
        print("ByteStrings Not Equal")

In [16]:
# if your function works properly, no error should appear when running this line
assert hexlify(xor_bytes(unhexlify(b'49207374696c6c206861766520584f522065796573206e6f772077686174'), unhexlify(b'3a4f1e11060209001b04180100302a3e5045141c5345170a040016292955'))) == b'736f6d656f6e652073656e642068656c70206d7920657965732061414821'

## 1-3. Single-byte XOR cipher [6 pts]

You could say that this is just the Caesar cipher but it works over binary strings instead. Anyways, I have encrypted the hex-encoded bytestring
```
264f0300190a4f0d06084f1c0a02061f1d06020a1c4f0e010b4f264f0c0e0101001b4f03060a4e
```
by XOR'ing it against a single character. Find out what key I used, and decrypt the message.

You can do this by hand (since there are only 256 possible keys to choose from), but don't!
**Write code to do this for you.**
Have some way of "scoring" a piece of English plaintext, like using character frequency.
So you can try each possible key and output the one with the best score.

Write a function called `try_decrypt_xor` that does just that, i.e., it takes a bytestring and outputs the key and the decrypted message (and optionally the score). And yes, you may implement your own functions as well.

In [17]:
# taken from http://pi.math.cornell.edu/~mec/2003-2004/cryptography/subs/frequencies.html
# feel free to use this!
FREQ_TABLE = {'e': 12.02, 't': 9.10, 'a': 8.12, 'o': 7.68, 'i': 7.31, 'n': 6.95, 
              's': 6.28, 'r': 6.02, 'h': 5.92, 'd': 4.32, 'l': 3.98, 'u': 2.88, 
              'c': 2.71, 'm': 2.61, 'f': 2.30, 'y': 2.11, 'w': 2.09, 'g': 2.03, 
              'p': 1.82, 'b': 1.49, 'v': 1.11, 'k': 0.69, 'x': 0.17, 'q': 0.11, 
              'j': 0.10, 'z': 0.07, ' ': 19.18}

In [18]:
def try_decrypt_xor(c):
    unhex_c = unhexlify(c)
    best_score = 0
    best_key = None
    msg = ""
    for i in range(0, 256):
        char = bytes(chr(i), 'latin_1')
        key = char * len(unhex_c)
        decode = xor_bytes(unhex_c, key)
        score = get_score(decode)
        if score >= best_score:
            best_score = score
            best_key = char
            msg = decode
    return {"score": best_score, "key": best_key, "msg": msg}

def create_ftable(txt):
    ftable = {}
    for i in 'abcdefghijklmnopqrstuvwxyz ':
        ftable[i] = 0
    for letter in txt:
        if chr(letter) in ftable:
            ftable[chr(letter)] += 1
    return ftable

def get_score(txt):
    score = 0
    freq = create_ftable(txt)
    for i in txt:
        if chr(i) in freq:
            score += freq[chr(i)] * FREQ_TABLE[chr(i)]
    return score


In [19]:
decrypt = try_decrypt_xor(b'264f0300190a4f0d06084f1c0a02061f1d06020a1c4f0e010b4f264f0c0e0101001b4f03060a4e')
print(f"key: {decrypt['key']}\nmessage: {decrypt['msg'].decode('utf-8')}\nscore: {decrypt['score']}")

key: b'o'
message: I love big semiprimes and I cannot lie!
score: 1454.9299999999998


## 1-4. Detect single-character XOR [6 pts]

I have a [file](https://gist.github.com/alltootechnical/bdb4f9fa751a5039fbbda8cfd5579e60) that contains 420 64-character hex-encoded strings, and **only one** of them has been encrypted by single-character XOR. 

Find that string.

*Hint:* Your code from the previous item should help.

In [20]:
fin = open('xor_strings.txt', 'rb')

In [21]:
strings = fin.readlines()

In [22]:
best_dec = {'score': 0, 'key': "", 'msg': ""}
for i in strings:
    dec = try_decrypt_xor(i[:-1])
    if dec['score'] >= best_dec['score']:
        best_dec = dec
        line = i
print(f"line: {i}\nmsg: {best_dec['msg'].decode('utf-8')}\nkey: {best_dec['key']}\nscore: {best_dec['score']}")

line: b'13d75f630f040788420340322e0437091ea01f6cc54592f3ff63937d0292a816\n'
msg: I found the hay in a needlestack
key: b'?'
score: 1131.73


## 1-5. Implement repeating-key XOR [4 pts]

Here are the first two lines of what people consider one of [Frank's greatest hits](https://youtu.be/ntULIFnj7MY):
```
Shawty had them apple bottom jeans,
The boots with the fur
```
Encrypt it using repeating-key XOR, with the key "`LOW`".

The way repeating-key XOR works is that the first byte of plaintext will be XOR'd against `L`, the next `O`, the next `W`, then `L` again for the 4th byte, and so on. In effect, it will look something like this:
```
Plaintext:   S h a w t y   h a d   t ...
Key:         L O W L O W L O W L O W ...
```
If this reminds you of the Vigenère cipher, you're absolutely right! This *is* just Vigenère, but we're working over binary strings instead.

Write a function called `xor_repeating` that takes two bytestrings corresponding to the plaintext and the key, and encrypts it using repeating-key XOR. Using the plaintext and key above, your function should output:
```
1f27363b3b2e6c2736286f23242a3a6c2e273c23326c2d38383b38216f
3d292e393f635d1827326c2d38233b246c383e3827773827326c29223e
```
when hex-encoded. (Note that this is a single hex string; it's only split into two lines to make it fit.)

In [23]:
def xor_repeating(m, key):
    full_key = (key*((len(m)//len(key)))*2)[:len(m)]
    return xor_bytes(m, full_key)

In [24]:
hexlify(xor_repeating(b'Shawty had them apple bottom jeans,\nThe boots with the fur', b'LOW'))

b'1f27363b3b2e6c2736286f23242a3a6c2e273c23326c2d38383b38216f3d292e393f635d1827326c2d38233b246c383e3827773827326c29223e'

In [25]:
# if your function works properly, no error should appear when running this line
assert hexlify(xor_repeating(b'Shawty had them apple bottom jeans,\nThe boots with the fur', b'LOW')) == b'1f27363b3b2e6c2736286f23242a3a6c2e273c23326c2d38383b38216f3d292e393f635d1827326c2d38233b246c383e3827773827326c29223e'

## 1-6. Break repeating-key XOR [10 pts]

I have [another file here](https://gist.github.com/alltootechnical/269eccf1b4d2f5e1512e248452e4ddc2).
It has been Base64-encoded after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let $n$ be the guessed key length. Try values from $n = 2$ up to $n = 40$.

2. Write a function called `hamming_dist` that computes the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) between two bytestrings of equal length.
The Hamming distance is just the number of differing bits.
For example, the Hamming distance between `twelve plus one` and `ElEveN pLUs twO` is $30$. **Please make sure your function works correctly first before proceeding.**

3. For each $n$, take the first $n$ bytes and the second $n$ bytes, and find the Hamming distance between them. 
Divide the result by $n$ to normalize it.

4. The $n$ with the smallest normalized Hamming distance is *probably* the key. But just to make sure, you could proceed with the smallest 2 or 3 values of $n$, or you could take 4 blocks of $n$ instead of 2 and average their distances.

5. Now that you probably know $n$, split the ciphertext into blocks of length $n$.

6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on. For example, suppose we have the ciphertext split into blocks of length 5:
```
QWERT YUIOP ASDFG HJKLZ
```
then transposing these blocks will yield five blocks of length 4:
```
QYAH WUSJ EIDK ROFL TPGZ
```

7. Solve each block as if it was single-character XOR. You have code to do this, so go use that.

8. For each block, the single-byte XOR key that has the best score is the repeating-key XOR key byte for that block. All you need to do is to put them together and you have the key!

As before, you may implement additional functions of your own.

In [26]:
def hamming_dist(a, b):
    dist = 0
    for i in list(xor_bytes(a, b)):
        dist += bin(i).count('1')
    return dist

In [27]:
# if your function works properly, no error should appear when running this line
assert hamming_dist(b'twelve plus one', b'ElEveN pLUs twO') == 30

In [28]:
fin = open('encrypted.txt', 'rb')

In [29]:
b64str = fin.read()

In [30]:
def decrypt_repeating_xor(cipher):
    decode64 = b64decode(cipher)
    key_length_lst = guess_key_length(decode64, 14)
    key, key_score = find_key(decode64, key_length_lst[:3])
    return key, key_score, xor_repeating(decode64, key)

In [31]:
def guess_key_length(cipher, b):
    ham_dict = {}
    blocks = b
    for key_length in range(2, 41):
        sum_ham = 0
        for i in range(0, (key_length*2)*blocks, key_length):
            bytes1 = cipher[i:i+key_length]
            bytes2 = cipher[i+key_length:i+key_length*2]
            sum_ham += hamming_dist(bytes1, bytes2)/key_length
        ave_ham = sum_ham/blocks
        ham_dict[key_length] = ave_ham
    sort_dict = sorted(ham_dict.items(), key = lambda x: x[1])
    return [x[0] for x in sort_dict]

In [32]:
def find_key(cipher, key_lengths):
    best_key_score = 0
    best_key = b''
    for k in key_lengths:
        chunks = []
        for x in range(0, len(cipher), k):
            chunks.append(cipher[x: x+k])
            
        transpose = [b'']*k
        for i in range(0, len(chunks)):
            for j in range(0, len(chunks[i])):
                transpose[j] += chunks[i][j:j+1]

        temp_key = b''
        for i in transpose:
            dec = try_decrypt_xor(hexlify(i))
            temp_key += dec['key']

        key_score = get_score(temp_key)
        if key_score >= best_key_score:
            best_key_score = key_score
            best_key = temp_key
        
        #print(f"key length: {k}, key: {temp_key}, score: {key_score}")
    return best_key, best_key_score

In [33]:
key, score, msg = decrypt_repeating_xor(b64str)
print(f"Best key: {key}\nKey Score: {score}\n\nMessage:\n{msg.decode('utf-8')}")

Best key: b'Bigger number, better person'
Key Score: 676.3000000000001

Message:
NARRATOR: Japan is an island by the sea filled with volcanoes and it's...

CHORUS: Beautiful!

NARRATOR: In the year -1,000,000,000... Japan might not have been here. In the year -40,000 it was here and you could walk to it. And some people walked to it.

The year is now -12,000

NARRATOR: Then it got warmer, some icebergs melted, it became an island, and now there's lots of trees because it's warmer.

The year is now -10,000

NARRATOR: So, now there's people on the island and they're basically sort of hanging out in between the mountains eating nuts off trees and using the latest technology. Like stones and bowls.

The sound of a doorbell chime. The year is now -500.

OUTSIDE WORLD (KOREA): Ding dong.

NARRATOR: It's the outside world, and they have technology from the future, like really good metal and crazy rice farms!
Now, you can make a lot of rice really, really quickly.
That means if you own the far